# Kayıp Fonksiyonları

Bu alıştırmada, Kayıp fonksiyonlarının `LinearRegression` modeli üzerindeki etkilerini karşılaştıracaksınız.

👇 Bu zorluk için kullanmak üzere bir CSV dosyası indirelim ve onu bir DataFrame'e dönüştürelim

In [5]:
import pandas as pd

data = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/loss_functions_dataset.csv")
data.sample(5)

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
277,0.66,759.5,318.5,220.50,3.5,0.10,13.075
1,0.98,514.5,294.0,110.25,7.0,0.00,18.440
389,0.90,563.5,318.5,122.50,7.0,0.25,33.750
673,0.98,514.5,294.0,110.25,7.0,0.40,32.865
179,0.69,735.0,294.0,220.50,3.5,0.10,12.770


🎯 Göreviniz, tasarımına göre bir seranın içindeki ortalama sıcaklığı tahmin etmektir. Sıcaklık tahminleriniz, her bir bitki için iklim ihtiyaçlarına göre uygun sera tasarımını seçmenize yardımcı olacaktır.

🌿 Bitkilerin küçük sıcaklık değişimlerini kaldırabildiğini, ancak sıcaklık değişimleri arttıkça katlanarak daha duyarlı hale geldiğini biliyorsunuz.

## 1. Teori

❓ Teorik olarak, bitkileri öldürme riskini sınırlamak için modelinizi hangi Kayıp fonksiyonu üzerinde eğitirsiniz?

<details>
<summary> 🆘 Cevap </summary>
    
Teorik olarak, Ortalama Kare Hata (MSE) Kayıp fonksiyonunu kullanırsınız. Bu, aykırı tahminleri cezalandırır ve modelinizin büyük hatalar yapmasını engeller. Bu, daha küçük sıcaklık değişimleri ve bitkiler için daha düşük risk sağlayacaktır.

</details>

MSE


## 2. Uygulama

### 2.1 Ön İşleme

In [6]:
data.shape


(768, 7)

❓ Özellikleri standartlaştırın

In [16]:
from sklearn.preprocessing import StandardScaler

#X=data.drop(columns=['Average Temperature'])
X=data.loc[:,'Relative Compactness':'Glazing Area']
y=data['Average Temperature']

scaler=StandardScaler().fit(X)
x_scaled=scaler.transform(X)
import numpy as np
x_scaled_df = pd.DataFrame(x_scaled, columns=X.columns)
print(x_scaled_df.head())


   Relative Compactness  Surface Area  Wall Area  Roof Area  Overall Height  \
0              2.041777     -1.785875  -0.561951  -1.470077             1.0   
1              2.041777     -1.785875  -0.561951  -1.470077             1.0   
2              2.041777     -1.785875  -0.561951  -1.470077             1.0   
3              2.041777     -1.785875  -0.561951  -1.470077             1.0   
4              1.284979     -1.229239   0.000000  -1.198678             1.0   

   Glazing Area  
0     -1.760447  
1     -1.760447  
2     -1.760447  
3     -1.760447  
4     -1.760447  


In [12]:
data.head()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
0,0.98,514.5,294.0,110.25,7.0,0.0,18.44
1,0.98,514.5,294.0,110.25,7.0,0.0,18.44
2,0.98,514.5,294.0,110.25,7.0,0.0,18.44
3,0.98,514.5,294.0,110.25,7.0,0.0,18.44
4,0.90,563.5,318.5,122.50,7.0,0.0,24.56


### 2.2 Modelleme

Bu bölümde, farklı Kayıp fonksiyonları üzerinde optimize edilmiş modelleri değerlendirerek teoriyi doğrulayacaksınız.

### En Küçük Kareler (MSE) Kaybı

❓ **En Küçük Kareler Kaybı** (MSE) üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

In [27]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_validate

sgd_model = SGDRegressor(loss="squared_error")

cv_results = cross_validate(sgd_model, 
                            x_scaled, 
                            
                            y,
                            cv=10,                              
                            scoring=['r2', 'neg_max_error'])
print(cv_results)

{'fit_time': array([0.00604892, 0.00445747, 0.00549054, 0.00654459, 0.00791717,
       0.00544143, 0.00566006, 0.00523043, 0.00483513, 0.00476789]), 'score_time': array([0.00165296, 0.00154781, 0.00180674, 0.0020256 , 0.00136423,
       0.00129533, 0.00175738, 0.00118279, 0.00117087, 0.00115561]), 'test_r2': array([0.78375318, 0.90748669, 0.89572328, 0.88415016, 0.9313138 ,
       0.8966634 , 0.92639484, 0.91621873, 0.89502748, 0.94036684]), 'test_neg_max_error': array([-9.88164274, -8.63092836, -8.76452249, -9.13400886, -8.87994555,
       -8.54452432, -8.52752259, -8.74244628, -8.39061583, -7.63343526])}


❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru ve bunu `r2` değişkeninde kaydedin
- Tüm katlarınızın °C cinsinden en büyük tek tahmin hatasını hesaplayın ve `max_error_celsius` değişkeninde kaydedin

(İpucu: `max_error` sklearn'de kabul edilen bir puanlama metriğidir)

In [46]:
r2 = cv_results['test_r2'].mean()
r2

0.8977098401154266

In [45]:
max_error_celsius = cv_results['test_neg_max_error'].min()
max_error_celsius

-9.88164273770322

### Ortalama Mutlak Hata (MAE) Kaybı

Peki modelimizi MAE üzerinde optimize edersek ne olur?

❓ **MAE** Kaybı üzerinde **Stokastik Gradyan İnişi** (SGD) ile optimize edilmiş bir Doğrusal Regresyon modelini **10-Katlı Çapraz doğrula**

<details>
<summary>💡 İpuçları</summary>

- MAE kaybı `SGDRegressor`'da doğrudan belirtilemez. Doğru parametreleri ayarlayarak tasarlanması gerekir

</details>

In [35]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_validate
import numpy as np

mae_model=SGDRegressor(loss='epsilon_insensitive',
                      epsilon=0,                    
                        max_iter=1000,
                             random_state=42)

cv_results_mae = cross_validate(mae_model, 
                                x_scaled, 
                                y,
                                cv=10,                              
                                scoring=['r2', 'neg_max_error'])

print(cv_results_mae)

{'fit_time': array([0.0087781 , 0.00709391, 0.0089159 , 0.00982928, 0.00798941,
       0.007406  , 0.00739908, 0.00812531, 0.00685716, 0.00763321]), 'score_time': array([0.00148821, 0.00156641, 0.00149298, 0.00153279, 0.00145936,
       0.00136876, 0.0012207 , 0.00115609, 0.00112939, 0.00108409]), 'test_r2': array([0.7432504 , 0.87564247, 0.8757397 , 0.84648879, 0.91754144,
       0.87354643, 0.9175109 , 0.8991184 , 0.87887026, 0.93556657]), 'test_neg_max_error': array([-11.06228474, -10.61762816, -10.62386046, -11.21001322,
       -11.13339995, -10.96225611, -10.81937682, -11.1413182 ,
       -10.92842515, -10.15643539])}


❓ Hesaplayın:
- Ortalama çapraz doğrulanmış R2 skoru, bunu `r2_mae`'de saklayın
- Tüm katlarınızın en büyük tek tahmin hatasını, bunu `max_error_mae`'de saklayın

In [38]:
r2_mae = cv_results_mae['test_r2'].mean()
r2_mae

0.8763275370982454

In [43]:
max_error_mae = abs(cv_results_mae['test_neg_max_error']).max()
max_error_mae

11.210013223184902

## 3. Sonuç

❓ Değerlendirdiğiniz modellerden hangisi göreviniz için en uygun görünüyor?

<details>
<summary> 🆘Cevap </summary>
    
İki model arasında ortalama çapraz doğrulanmış r2 skorları yaklaşık olarak benzer olmasına rağmen, MAE üzerinde optimize edilen modelin zaman zaman daha büyük hatalar yapma şansı daha fazladır, bu da bitkileri öldürme riskini artırır!
    
</details>

> Mse çünkü aralarında 1.33 dereceye yakın fark var mse de aşırı sıcak dalgalanmalardan kaçınıyor r2 skoruda küçük farkta olsa daha yüksek

# 🏁 Kodunuzu kontrol edin ve notebook'unuzu gönderin

In [47]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'loss_functions',
    r2 = r2,
    r2_mae = r2_mae,
    max_error = max_error_celsius,
    max_error_mae = max_error_mae
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/funny/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/funny/S16D4-S-loss-functions/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 3 items

test_loss_functions.py::TestLossFunctions::test_max_error_order PASSED   [ 33%]
test_loss_functions.py::TestLossFunctions::test_r2 PASSED                [ 66%]
test_loss_functions.py::TestLossFunctions::test_r2_mae PASSED            [100%]

============================== 3 passed in 0.10s ===============================


💯 You can commit your code:

git add tests/loss_functions.pickle

git commit -m 'Completed loss_functions step'

git push origin master

